In [2]:
%pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00


In [23]:
%pip install networkx


In [3]:
import torch_geometric

In [4]:
from torch_geometric.datasets import TUDataset

In [5]:
dataset = TUDataset(root='/tmp/NCI1', name='NCI1')

Processing...
Done!


In [118]:
import numpy as np # type: ignore

class Softmax():
    def forward(self, A):
        exp = np.exp(A)
        return exp / np.sum(exp, axis=1, keepdims=True)

    def backward(self, A, Y):
        return A - Y


class SumPool():
    def forward(self, batch_indices, B, H_in):
        H_flattened = np.zeros(shape=(B, H_in.shape[-1]))
        np.add.at(H_flattened, batch_indices, H_in)
        return H_flattened

    def backward(self, batch_indices, dH_flattened):
        return dH_flattened[batch_indices]


class Linear():
    def __init__(self, in_features, out_features, beta1=0.9, beta2=0.98, learning_rate=1e-3, epsilon=1e-7):
        self.beta1 = beta1
        self.beta2 = beta2
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.W = np.random.normal(loc=0, scale=np.sqrt(2. / in_features), size=(out_features, in_features)) # He initialization
        self.b = np.zeros(shape=(1, out_features))
        self.VdW = np.zeros_like(self.W)
        self.Vdb = np.zeros_like(self.b)
        self.SdW = np.zeros_like(self.W)
        self.Sdb = np.zeros_like(self.b)

    def forward(self, A, use_relu=True):
        Z = np.matmul(A, self.W.T) + self.b
        if use_relu:
            return Z, np.maximum(Z, 0)
        else:
            return Z

    def backward(self, A, dA, Z=None, used_relu=True):
        # Calculate weight and bias gradients
        if not used_relu: # next layer was softmax --> dZ is none
            dZ = dA
        else: # next layer was not softmax but relu instead --> dA is none
            dZ = dA * np.where(Z > 0, 1, 0)
        dW = (1. / A.shape[0]) * np.matmul(dZ.T, A)
        db = (1. / A.shape[0]) * np.sum(dZ, axis=0, keepdims=True)
        dAprev = np.matmul(dZ, self.W)
        # Update Adam gradients
        self.VdW = self.beta1 * self.VdW + (1. - self.beta1) * dW
        self.Vdb = self.beta1 * self.Vdb + (1. - self.beta1) * db
        self.SdW = self.beta2 * self.SdW + (1. - self.beta2) * (dW ** 2)
        self.Sdb = self.beta2 * self.Sdb + (1. - self.beta2) * (db ** 2)
        # Update weights and biases
        self.W -= self.learning_rate * self.VdW / (np.sqrt(self.SdW) + self.epsilon)
        self.b -= self.learning_rate * self.Vdb / (np.sqrt(self.Sdb) + self.epsilon)
        return dAprev


class GCN():
    def __init__(self, d_in, d_out, beta1=0.9, beta2=0.98, alpha=1e-3, epsilon=1e-7):
        self.W = np.random.normal(loc=0, scale=2./(d_in * d_out), size=(d_out, d_in))
        self.b = np.zeros(shape=(1, d_out))
        self.beta1 = beta1
        self.beta2 = beta2
        self.alpha = alpha
        self.epsilon = epsilon
        self.VdW = np.zeros_like(self.W)
        self.SdW = np.zeros_like(self.W)
        self.Vdb = np.zeros_like(self.b)
        self.Sdb = np.zeros_like(self.b)

    def forward(self, A_hat, H):
        H_prime = np.matmul(A_hat, H)
        H_pre = np.matmul(H_prime, self.W.T) + self.b
        return H_prime, H_pre, np.maximum(H_pre, 0)

    def backward(self, A_hat, H_prime, H_pre, dHnext):
        # calculate gradients
        dH_pre = dHnext * np.where(H_pre > 0, 1, 0)
        dH_prime = np.matmul(dH_pre, self.W)
        dW = np.matmul(dH_pre.T, H_prime)
        db = np.sum(dH_pre, axis=0, keepdims=True)
        dH = np.matmul(A_hat.T, dH_prime)
        # update Adam gradients
        self.VdW = self.beta1 * self.VdW + (1. - self.beta1) * dW
        self.Vdb = self.beta1 * self.Vdb + (1. - self.beta1) * db
        self.SdW = self.beta2 * self.SdW + (1. - self.beta2) * (dW ** 2)
        self.Sdb = self.beta2 * self.Sdb + (1. - self.beta2) * (db ** 2)
        # update weights and biases
        self.W -= self.alpha * self.VdW / (np.sqrt(self.SdW) + self.epsilon)
        self.b -= self.alpha * self.Vdb / (np.sqrt(self.Sdb) + self.epsilon)
        return dH


In [119]:
class GNN():
    def __init__(self):
        self.gcn1 = GCN(d_in=34, d_out=64)
        self.gcn2 = GCN(d_in=64, d_out=64)
        self.sum_pool = SumPool()
        self.linear1 = Linear(in_features=64, out_features=32)
        self.linear2 = Linear(in_features=32, out_features=2)
        self.softmax = Softmax()

    def forward(self, batch_indices, batch_size, A_hat, H):
        H_prime1, H_pre1, H1 = self.gcn1.forward(A_hat, H)
        # print("H1: ", H1.shape)
        H_prime2, H_pre2, H2 = self.gcn2.forward(A_hat, H1)
        # print("H2: ", H2.shape)
        H_graph = self.sum_pool.forward(batch_indices, batch_size, H2)
        # print("H_graph: ", H_graph.shape)
        Z1, A1 = self.linear1.forward(H_graph, use_relu=True)
        # print("A1: ", A1.shape)
        A2 = self.linear2.forward(A1, use_relu=False)
        # print("A2: ", A2.shape)
        A3 = self.softmax.forward(A2)
        # print("A3: ", A3.shape)
        return (H_prime1, H_pre1, H1), (H_prime2, H_pre2, H2), (H_graph,), (Z1, A1), (A2,), (A3,)

    def backward(self, output_tuples, batch_indices, A_hat, H, Y):
        (H_prime1, H_pre1, H1), (H_prime2, H_pre2, H2), (H_graph,), (Z1, A1), (A2,), (A3,) = output_tuples
        dA2 = A3 - Y
        # print("dA2: ", dA2.shape)
        dA1 = self.linear2.backward(A1, dA2, used_relu=False)
        # print("dA1: ", dA1.shape)
        dH_graph = self.linear1.backward(H_graph, dA1, Z1, used_relu=True)
        # print("dH_graph: ", dH_graph.shape)
        dH2 = self.sum_pool.backward(batch_indices, dH_graph)
        # print("dH2: ", dH2.shape)
        dH1 = self.gcn2.backward(A_hat, H_prime2, H_pre2, dH2)
        # print("dH1: ", dH1.shape)
        dH = self.gcn1.backward(A_hat, H_prime1, H_pre1, dH1)
        # print("dH: ", dH.shape)




In [9]:
def cross_entropy_loss(A, Y):
    loss =  (-1. / A.shape[0]) * np.sum(Y * np.log(np.clip(A, 1e-7, 1-1e-7) + 1e-7), axis=1)
    return np.mean(loss)

In [26]:
import networkx as nx
G = nx.karate_club_graph()
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(list(G.nodes))
print(list(G.edges))

Number of nodes: 34
Number of edges: 78
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 12), (0, 13), (0, 17), (0, 19), (0, 21), (0, 31), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30), (2, 3), (2, 7), (2, 8), (2, 9), (2, 13), (2, 27), (2, 28), (2, 32), (3, 7), (3, 12), (3, 13), (4, 6), (4, 10), (5, 6), (5, 10), (5, 16), (6, 16), (8, 30), (8, 32), (8, 33), (9, 33), (13, 33), (14, 32), (14, 33), (15, 32), (15, 33), (18, 32), (18, 33), (19, 33), (20, 32), (20, 33), (22, 32), (22, 33), (23, 25), (23, 27), (23, 29), (23, 32), (23, 33), (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (27, 33), (28, 31), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33), (32, 33)]


In [27]:
for node in G.nodes(data=True):
    print(node)  # (node_id, {'club': faction})

(0, {'club': 'Mr. Hi'})
(1, {'club': 'Mr. Hi'})
(2, {'club': 'Mr. Hi'})
(3, {'club': 'Mr. Hi'})
(4, {'club': 'Mr. Hi'})
(5, {'club': 'Mr. Hi'})
(6, {'club': 'Mr. Hi'})
(7, {'club': 'Mr. Hi'})
(8, {'club': 'Mr. Hi'})
(9, {'club': 'Officer'})
(10, {'club': 'Mr. Hi'})
(11, {'club': 'Mr. Hi'})
(12, {'club': 'Mr. Hi'})
(13, {'club': 'Mr. Hi'})
(14, {'club': 'Officer'})
(15, {'club': 'Officer'})
(16, {'club': 'Mr. Hi'})
(17, {'club': 'Mr. Hi'})
(18, {'club': 'Officer'})
(19, {'club': 'Mr. Hi'})
(20, {'club': 'Officer'})
(21, {'club': 'Mr. Hi'})
(22, {'club': 'Officer'})
(23, {'club': 'Officer'})
(24, {'club': 'Officer'})
(25, {'club': 'Officer'})
(26, {'club': 'Officer'})
(27, {'club': 'Officer'})
(28, {'club': 'Officer'})
(29, {'club': 'Officer'})
(30, {'club': 'Officer'})
(31, {'club': 'Officer'})
(32, {'club': 'Officer'})
(33, {'club': 'Officer'})


In [28]:
labels = [0 if G.nodes[n]['club'] == 'Mr. Hi' else 1 for n in G.nodes()]
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
for u, v in G.edges():
    G[u][v].clear()  # Remove any attributes
A = nx.to_numpy_array(G)
print(A.shape)  # (34, 34)
print(A[:5, :5])  # First 5x5 block

(34, 34)
[[0. 1. 1. 1. 1.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [34]:
print(np.eye(34))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [41]:
A += np.eye(34)
D_inv_sqrt = np.diag(1. / np.sqrt(np.diag(np.sum(A, axis=1)).diagonal() + 1e-10))
A_hat = np.matmul(np.matmul(D_inv_sqrt, A), D_inv_sqrt)
print(A_hat[:5, :5])


[[0.27272727 0.05504819 0.05330018 0.06154575 0.07106691]
 [0.05504819 0.4        0.06454972 0.0745356  0.        ]
 [0.05330018 0.06454972 0.375      0.07216878 0.        ]
 [0.06154575 0.0745356  0.07216878 0.5        0.        ]
 [0.07106691 0.         0.         0.         0.66666667]]


In [48]:
H = np.eye(34) # treat each node as its own feature, since this dataset doesn't have any incorporated node features
print(np.array(G.edges()).shape)

(78, 2)


In [120]:
model = GNN()
batch_indices = np.zeros(shape=(34,), dtype=int)
label = np.array([[0,1]])

In [115]:
output_tuples = model.forward(batch_indices, 1, A_hat, H)
model.backward(output_tuples, batch_indices, A_hat, H, label)
print(output_tuples[-1][0])

H1:  (34, 64)
H2:  (34, 64)
H_graph:  (1, 64)
A1:  (1, 32)
A2:  (1, 2)
A3:  (1, 2)
dA2:  (1, 2)
dA1:  (1, 32)
dH_graph:  (1, 64)
dH2:  (34, 64)
dH1:  (34, 64)
dH:  (34, 34)
[[0.50000399 0.49999601]]


In [121]:
for _ in range(20):
  output_tuples = model.forward(batch_indices, 1, A_hat, H)
  model.backward(output_tuples, batch_indices, A_hat, H, label)
  print("OUTPUT: ", output_tuples[-1][0])
  print("LABEL: ", label)

OUTPUT:  [[0.49999782 0.50000218]]
LABEL:  [[0 1]]
OUTPUT:  [[0.48247982 0.51752018]]
LABEL:  [[0 1]]
OUTPUT:  [[0.45539582 0.54460418]]
LABEL:  [[0 1]]
OUTPUT:  [[0.42113153 0.57886847]]
LABEL:  [[0 1]]
OUTPUT:  [[0.37663024 0.62336976]]
LABEL:  [[0 1]]
OUTPUT:  [[0.32146436 0.67853564]]
LABEL:  [[0 1]]
OUTPUT:  [[0.25978683 0.74021317]]
LABEL:  [[0 1]]
OUTPUT:  [[0.19530966 0.80469034]]
LABEL:  [[0 1]]
OUTPUT:  [[0.135444 0.864556]]
LABEL:  [[0 1]]
OUTPUT:  [[0.08414489 0.91585511]]
LABEL:  [[0 1]]
OUTPUT:  [[0.04701864 0.95298136]]
LABEL:  [[0 1]]
OUTPUT:  [[0.02418126 0.97581874]]
LABEL:  [[0 1]]
OUTPUT:  [[0.01156778 0.98843222]]
LABEL:  [[0 1]]
OUTPUT:  [[0.00527271 0.99472729]]
LABEL:  [[0 1]]
OUTPUT:  [[0.00234406 0.99765594]]
LABEL:  [[0 1]]
OUTPUT:  [[0.0010332 0.9989668]]
LABEL:  [[0 1]]
OUTPUT:  [[4.57677749e-04 9.99542322e-01]]
LABEL:  [[0 1]]
OUTPUT:  [[2.06170272e-04 9.99793830e-01]]
LABEL:  [[0 1]]
OUTPUT:  [[9.51556607e-05 9.99904844e-01]]
LABEL:  [[0 1]]
OUTPUT:  [[4.